# Dubin's 2D Aircraft

Dubin's aircraft presents a dynamically simple 2D aircraft model. The state space is 3 dimensions, being two position coordinate $(x,y)$
and a heading angle $\theta$. The control action simply is to apply a heading angle $\theta$ while maintaining constant velocity
(no throttle). The update equation is

 \begin{equation}
  \dot{\mathbf x} = \begin{bmatrix}
v \cos (x_2) \\
v \sin (x_2) \\
u \\
\end{bmatrix}
\end{equation}

where $v$ is some fixed airspeed parameter.


## Controller Design

A lateral rejoin task is specified: given **n** planes at different orientations, produce **n** maneuver sequences that allow them to be no further
than some terminal length apart $r_l$ and at some terminal heading angle $\theta_t$. Given that the only control surface that can be affected is angular
rate, the following control scheme is formulated,

1.  Construct an undirected graph of planes, like the can satellite design above. In this case, a simple $k$-neighbors was done,
    with $k=1$. In this procedure, solve only for the nearest neighbor; this can be extended to different graph constructions, by
    appropriately weighting the graph edges inversely to distances between nodes.
2.  Solve for the angle that will cause a plane $s_i$ and its nearest neighbor $s_j$ to approach one another the fastest,
    $$\theta_{i}(s_i, s_j) = \operatorname{atan2}(x_{j1} -x_{i1}, x_{j0} - x_{i0})$$
3.  Linearly combine the angle $\theta_i$ and the terminal heading angle \(\theta_t\).
    Apply some weight that is a function of the distance between the aircraft $r$, $w: \bar{\mathbb R^-}  \rightarrow [0, 1]$,
    $$\theta_c(s_i) = w(r(s_i, s_j)) \theta_t + (1-w(r(s_i, s_j))) \theta_j$$
    In this example,
    $$w(r) = \exp\left( -\frac{(r-r_l)^2}{\tau} \right)$$.
4.  Control the position quantity via a proportional controller,
    $$u = k_p (\theta_c - \theta)$$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import csaf.config as cconf
import csaf.system as csys

# create a csaf configuration out of toml
my_conf = cconf.SystemConfig.from_toml("/csaf-system/rejoin_config.toml")

In [ ]:
def set_dub_idx(conf, identifier, number):
    conf.config_dict['components'][identifier+str(number)]['config']['parameters']['idx']=number
    
def set_dub_state(conf, identifier, number, state):
    my_conf.config_dict['components'][identifier+str(number)]['config']['topics']['states']['initial'] = state

In [ ]:
states = [[0, 0, np.deg2rad(45)],
              [-5, -10, np.deg2rad(-30)],
            [-3, -15, np.deg2rad(90)],
            [0, -20, np.deg2rad(0)]]

for idx, state in enumerate(states):
    set_dub_idx(my_conf, 'jet', idx)
    set_dub_state(my_conf, 'jet', idx, state)

In [ ]:
from IPython.display import Image

import pathlib

plot_fname = f"pub-sub-plot.png"

# plot configuration pub/sub diagram as a file -- proj specicies a dot executbale and -Gdpi is a valid dot
# argument to change the image resolution
my_conf.plot_config(fname=pathlib.Path(plot_fname).resolve(), prog=["dot", "-Gdpi=400"])

# display written file to notebook
Image(plot_fname, height=600)

In [ ]:
# create pub/sub components out of the configuration
my_system = csys.System.from_config(my_conf)

simulation_timespan = [0, 25.0]

# simulate and collect time traces out of the components
trajs = my_system.simulate_tspan(simulation_timespan, show_status=True)

# destroy components and unbind all used sockets
my_system.unbind() 

In [ ]:
states = [np.array(trajs[f"jet{idx}"].states) for idx in range(4)]

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
from matplotlib.colors import LinearSegmentedColormap

fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlabel('X (m)', size = 12)
ax.set_ylabel('Y (m)', size = 12)
ax.set_xlim([-200, 200])
ax.set_ylim([-250, 100])
ax.set_aspect('equal')
#ax.axis([0,1,0,1])
x_vals = []
y_vals = []
intensity = []
iterations = 100

t_vals = np.linspace(0,1, iterations)

colors = [[0,0,1,0],[0,0,1,0.5],[0,0.2,0.4,1]]
cmap = LinearSegmentedColormap.from_list("", colors)
scatter = ax.scatter(x_vals,y_vals, c=[], cmap=cmap, vmin=0,vmax=1, s=5, label="Chaser Satellites")
#ax.scatter(0, 0, c='r', label="Chief Satellite")
#a_circle = plt.Circle((0, 0), 3.0, color='k', fill=False)
#ax.add_artist(a_circle)
#ax.legend()

def get_new_vals(i):
    n = np.random.randint(1,5)
    #x = np.random.rand(n)
    #y = np.random.rand(n)
    x = [s[i, 0]*10 for s in states]
    y = [s[i, 1]*10 for s in states]
    return list(x), list(y)

def update(t):
    global x_vals, y_vals, intensity
    # Get intermediate points
    new_xvals, new_yvals = get_new_vals(t)
    x_vals.extend(new_xvals)
    y_vals.extend(new_yvals)
    
    # Put new values in your plot
    scatter.set_offsets(np.c_[x_vals,y_vals])

    #calculate new color values
    intensity = np.concatenate((np.array(intensity)*0.96, np.ones(len(new_xvals))))
    scatter.set_array(intensity)
    xb = [np.min(np.concatenate([s[t-100:, 0] for s in states])), 
                    np.max(np.concatenate([s[t-100:, 0] for s in states]))]
    yb = [np.min(np.concatenate([s[t-100:, 1] for s in states])), 
                    np.max(np.concatenate([s[t-100:, 1] for s in states]))]
    if t > 100:
        ax.set_xlim(xb[0]*10 - 0.1 * np.abs(xb[0]*10), xb[1]*10 + 0.1 * np.abs(xb[1]*10))
        ax.set_ylim(yb[0]*10 - 0.1 * np.abs(yb[0]*10), yb[1]*10 + 0.1 * np.abs(yb[1]*10))

    # Set title
    ax.set_title('Dubin\'s 2D Plane Rejoin (Epoch: %d)' %t)

ani = matplotlib.animation.FuncAnimation(fig, update, frames=300,interval=50)
from IPython.display import HTML
HTML(ani.to_jshtml())
#plt.show()